In [ ]:
from google.colab import files
files.upload()

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c dogs-vs-cats

 99% 801M/812M [00:03<00:00, 223MB/s]
100% 812M/812M [00:04<00:00, 212MB/s]


In [4]:
!unzip -qq dogs-vs-cats.zip
!unzip -qq train.zip

In [5]:
import os, shutil, pathlib

o_dir = pathlib.Path("train")
n_dir = pathlib.Path("dogs_vs_cats_small")

def make_subsets(name, start, end):
    for category in ("dog", "cat"):
        dir = n_dir / name / category
        os.makedirs(dir)
    
        fnames = [f"{category}.{i}.jpg" for i in range(start, end)]

        for fname in fnames:
            shutil.copyfile(src=o_dir / fname, dst=dir / fname)

make_subsets("train", start=0, end=1_000)
make_subsets("validation", start=1_000, end=1_500)
make_subsets("test", start=1_500, end=2_500)

In [6]:
from tensorflow.keras.utils import image_dataset_from_directory

train_data = image_dataset_from_directory(
    n_dir / "train",
    image_size=(180, 180),
    batch_size=32
)

val_data = image_dataset_from_directory(
    n_dir / "validation",
    image_size=(180, 180),
    batch_size=32
)

test_data = image_dataset_from_directory(
    n_dir / "test",
    image_size=(180, 180),
    batch_size=32
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [19]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import (
    Conv2D, 
    Dense, 
    Input, 
    MaxPooling2D,
    Flatten,
    Rescaling,
    BatchNormalization,
    Activation,
    RandomRotation,
    RandomFlip,
    RandomZoom,
    Dropout
)

data_aug = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.2)
])

inputs = Input(shape=(180, 180, 3))
x = data_aug(inputs)
x = Rescaling(1. /255)(x)

x = Conv2D(32, 3)(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=2)(x)

x = Conv2D(64, 3)(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=2)(x)

x = Conv2D(128, 3)(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=2)(x)

x = Conv2D(256, 3)(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=2)(x)

x = Conv2D(256, 3)(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=2)(x)

x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=RMSprop(learning_rate=1e-5),
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 sequential_5 (Sequential)   (None, 180, 180, 3)       0         
                                                                 
 rescaling_5 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_25 (Conv2D)          (None, 178, 178, 32)      896       
                                                                 
 batch_normalization_20 (Bat  (None, 178, 178, 32)     128       
 chNormalization)                                                
                                                                 
 activation_25 (Activation)  (None, 178, 178, 32)      0         
                                                           

In [20]:
from tensorflow.keras.callbacks import ModelCheckpoint

callbacks = [
    ModelCheckpoint(
        montior="validation_loss",
        save_best_only=True,
        filepath="cnn.keras"
    )
]

In [21]:
model.fit(train_data,
          epochs=50,
          validation_data=val_data,
          callbacks=callbacks)

Epoch 1/50
63/63 [==============================] - 8s 91ms/step - loss: 0.9650 - accuracy: 0.5190 - val_loss: 0.6941 - val_accuracy: 0.5000
Epoch 2/50
63/63 [==============================] - 5s 79ms/step - loss: 0.8962 - accuracy: 0.5555 - val_loss: 0.7070 - val_accuracy: 0.5000
Epoch 3/50
63/63 [==============================] - 5s 79ms/step - loss: 0.8963 - accuracy: 0.5625 - val_loss: 0.7348 - val_accuracy: 0.5000
Epoch 4/50
63/63 [==============================] - 5s 80ms/step - loss: 0.8461 - accuracy: 0.5795 - val_loss: 0.7701 - val_accuracy: 0.5010
Epoch 5/50
63/63 [==============================] - 5s 80ms/step - loss: 0.8179 - accuracy: 0.5845 - val_loss: 0.7904 - val_accuracy: 0.5000
Epoch 6/50
63/63 [==============================] - 5s 78ms/step - loss: 0.8162 - accuracy: 0.5970 - val_loss: 0.7473 - val_accuracy: 0.5180
Epoch 7/50
63/63 [==============================] - 5s 80ms/step - loss: 0.8120 - accuracy: 0.6060 - val_loss: 0.6720 - val_accuracy: 0.5970
Epoch 8/50
63

In [22]:
model.evaluate(test_data)

63/63 [==============================] - 3s 36ms/step - loss: 0.5190 - accuracy: 0.7480


[0.5190333127975464, 0.7480000257492065]